In [1]:
from langchain.text_splitter import Language
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser

In [2]:
repo_path = "C:\\Users\\rutvi\\Downloads\\ff"

In [3]:
loader = GenericLoader.from_filesystem(
    repo_path ,
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)
documents = loader.load()
len(documents)

2

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

2

In [9]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain import OpenAI
import openai
# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

In [10]:
#Stuffing method

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k"")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
# print(docs)
answers = stuff_chain.run(docs)


# # Print or use stored_answers as needed
answers.split("\n\n")
type(answers)
print(answers)
# print(stuff_chain.run(docs))

C:\Users\rutvi\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The first code snippet is a method that paints a logo on a graphics object. It checks if the logo is null, and if not, it calculates the position and size of the logo and draws it on the graphics object.

The second code snippet is a method that centers a component on the screen. It uses the screen size and the component's dimensions to calculate the position and sets the location of the component accordingly.


In [34]:
#Method 2 Map-Reduce
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain

llm = ChatOpenAI(temperature=0")

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [35]:
from langchain import hub
obj = hub.pull("rlm/map-prompt")

In [36]:
obj

ChatPromptTemplate(input_variables=['docs'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], template='The following is a set of documents:\n{docs}\nBased on this list of docs, please identify the main themes \nHelpful Answer:'))])

In [37]:
# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [38]:
# Note we can also get this from the prompt hub, as noted above
reduce_prompt = hub.pull("rlm/map-prompt")

In [40]:
from langchain.prompts.chat import ChatPromptTemplate

In [41]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

In [42]:
ChatPromptTemplate(input_variables=['docs'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], template='The following is a set of documents:\n{docs}\nBased on this list of docs, please identify the main themes \nHelpful Answer:'))])

ChatPromptTemplate(input_variables=['docs'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], template='The following is a set of documents:\n{docs}\nBased on this list of docs, please identify the main themes \nHelpful Answer:'))])

In [43]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [64]:
from langchain_text_splitters import CharacterTextSplitter


In [66]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [67]:
print(map_reduce_chain.run(split_docs))

The main themes identified in these documents are painting or displaying a logo/icon using graphics in a software application and centering a component on the screen.


In [70]:
#MEthod 3
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="refine")
chain.run(split_docs)

'The code snippet defines a method to paint a logo on a graphics object. If the logo is null, the method returns. The logo is then drawn at a specific position on the graphics object. The comment indicates that the method paints the icon. Additionally, a method is provided to center a component on the screen.'

In [73]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in gujarati"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

In [74]:
print(result["output_text"])

મેથડ paintLogo ને વધુ સંદર્ભ સાથે સુધારો કરવાની આવશ્યકતા નથી, તેની મૂળ સારાંશ પરંતુ પૂર્ણ છે: જો લોગો આઇકોન નલ હોય, તો તે નિર્દિષ્ટ પહોળા અને ઊંચાઈનો ઉપયોગ કરીને સ્ક્રીન પર લોગો આઇકોન પેઇન્ટ કરે છે.


In [75]:
print("\n\n".join(result["intermediate_steps"][:3]))

The paintLogo method paints the logo icon on the screen if it is not null, using the specified width and height.

મેથડ paintLogo ને વધુ સંદર્ભ સાથે સુધારો કરવાની આવશ્યકતા નથી, તેની મૂળ સારાંશ પરંતુ પૂર્ણ છે: જો લોગો આઇકોન નલ હોય, તો તે નિર્દિષ્ટ પહોળા અને ઊંચાઈનો ઉપયોગ કરીને સ્ક્રીન પર લોગો આઇકોન પેઇન્ટ કરે છે.


In [77]:
###end####

In [ ]:
st.title("Magical Code Summarizer 🧙‍♂️✨")
prompt = st.chat_input("Code Summarizer")
# Initialize chat history if necessary
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []


def display_chat_history():
    for entry in st.session_state.chat_history:
        if entry["role"] == "user":
            st.markdown(f"**You 🧙‍♂️ :** {entry['text']}")
        else:
            st.markdown(f"**Code Summarizer 📝:** {entry['text']}")


In [10]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"")
chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(docs)

'The code snippets show methods in Java for painting a logo and centering a component on the screen. The paintLogo method paints a logo on a graphics object, while the center method centers a component on the screen using the default screen size and component dimensions.'

In [11]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"")
chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(docs)

'The code snippets show methods in Java for painting a logo and centering a component on the screen. The paintLogo method paints a logo on a graphics object, while the center method centers a component on the screen using the screen and component dimensions.'